In [1]:
import pandas as pd
import numpy as np
import html
import nltk
import regex as re
import string
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [111]:
data = pd.read_csv('C:/Users/Danilo/Desktop/Documentos Acadêmicos/TERA/TERAprojetos/Projeto final/BNCC/repo/bncc-classifier/data/raw/bncc_first_classifier.csv')

In [112]:
data_clone = data.copy()

In [113]:
# renomeando as colunas do dataset

data_clone.columns = ['id', 'questoes', 'tipoQuestoes', 'topico', 'slug', 'materia', 'slug.1','etapaEnsino']
data_clone.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II


# EDA

In [27]:
data_clone.shape

(100000, 8)

In [76]:
data_clone['etapaEnsino'].value_counts().to_frame()

,etapaEnsino
Médio & Pré-Vestibular,35676
Fundamental II,33790
Fundamental I,18379
Concurso,9875
Militar,2200
OAB,80


Selecionando apenas questões relacionadas ao Ensino Fundamental e Médio

In [114]:
data_step_clean_degree = data_clone[data_clone["etapaEnsino"].isin(["Médio & Pré-Vestibular", "Fundamental II", "Fundamental I"])]
data_step_clean_degree.shape

(87845, 8)

In [78]:
data_step_clean_degree[data_step_clean_degree["materia"] == 'Produção de Texto'].sample(5)

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino
96676,3897924,<p>3 &ndash; DE ACORDO COM O TEXTO QUE ESTUDAM...,2,Outros,outros,Produção de Texto,producao-de-texto,Fundamental I
57374,2946062,<p> P&aacute;gina 6</p>,4,Outros,outros,Produção de Texto,producao-de-texto,Fundamental II
57390,1358332,"<p> No texto 5, em: “Holmes sempre foi o homem...",2,Outros,outros,Produção de Texto,producao-de-texto,Fundamental II
57104,2938035,<p><strong>NÃO</strong> é característica do ar...,2,Outros,outros,Produção de Texto,producao-de-texto,Fundamental II
87506,1189992,<p> Fotografia da tarefa - caderno de atividad...,4,Outros,outros,Produção de Texto,producao-de-texto,Fundamental II


In [115]:
data_step_clean_degree["materia"].value_counts(ascending=True)

Língua inglesa              1
Geral                       2
Conhecimentos Gerais        7
Italiano                   11
Francês                    17
Alemão                     25
Ciências Humanas           33
Desenho Geométrico         53
Atualidades                70
Produção de Texto         371
Ensino Religioso          468
Redação                  1035
Sociologia               1392
Outros                   1522
Literatura               2199
Espanhol                 2278
Educação Física          2349
Biologia                 3041
Química                  3108
Arte                     3681
Filosofia                3714
Física                   5366
Ciências                 7095
Inglês                   7968
História                 8913
Geografia                9111
Língua Portuguesa       11887
Matemática              12128
Name: materia, dtype: int64

- Agrupando categorias

In [116]:
data_step_clean_subjects = data_step_clean_degree.copy()

data_step_clean_subjects.replace('Língua inglesa','Inglês', inplace = True)
data_step_clean_subjects.replace('Conhecimentos Gerais','Geral', inplace = True)
data_step_clean_subjects.replace('Desenho Geométrico','Matemática', inplace = True)
data_step_clean_subjects.replace(['Produção de Texto', 'Redação', 'Literatura'],'Língua Portuguesa', inplace = True)

- Excluindo categorias

In [117]:
data_step_clean_subjects = data_step_clean_subjects[~data_step_clean_subjects["materia"].isin([
    'Italiano', 'Francês','Alemão', 'Atualidades','Espanhol','Geral', 'Outros', 'Ciências Humanas'])]
data_step_clean_subjects.shape

(83880, 8)

In [108]:
data_step_clean_subjects['materia'].value_counts()

Língua Portuguesa    15492
Matemática           12181
Geografia             9111
História              8913
Inglês                7969
Ciências              7095
Física                5366
Filosofia             3714
Arte                  3681
Química               3108
Biologia              3041
Educação Física       2349
Sociologia            1392
Ensino Religioso       468
Name: materia, dtype: int64

### Criando categorias reduzidas

In [ ]:
data_step_clean_main_areas = data_step_clean_subjects.copy()

data_step_clean_main_areas.replace('Língua inglesa','Inglês', inplace = True)

### Tipo questão

A variável tipoQuestoes contém items dos tipos:

1. Somatória
2. Múltipla Escolha
3. Certo e Errado
4. Discursiva
5. Redação ou items que é necessário enviar foto da resolução
6. Questões com mais de uma alternativa certa
7. Não representa nada em específico

Iremos remover questoes tipo 5 e 6 pois não se adequam ao tipo de alvo do nosso problema.

In [94]:
data_step_clean_subjects["tipoQuestoes"].value_counts(ascending=False).to_frame()

,tipoQuestoes
2,65544
4,13282
1,2860
3,1044
5,978
7,201
6,4


Obs.: Questoes tipo Multipla escolha e Discursiva somam a maioria das questões

### Remover as questoes tipo 6 e 7

In [118]:
data_step_clean_ques = data_step_clean_subjects[~data_step_clean_subjects["tipoQuestoes"].isin([6, 7])]
data_step_clean_ques.shape

(83675, 8)

In [84]:
data_step_clean_ques.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II
5,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,4,Proposta,proposta,Língua Portuguesa,redacao,Fundamental I
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,4,Idade Moderna: Revolução Francesa: Antecedente...,idade-moderna-revolucao-francesa-antecedentes-...,História,historia,Fundamental II


In [106]:
data_step_clean_ques["topico"].value_counts(ascending=False).to_frame()

,topico
Outros,24114
Vocabulary,1122
Gêneros textuais: Interpretação,894
Geral,682
Text Comprehension,607
...,...
Matéria e energia: Fontes e tipos de energia: Tipos de usinas geradoras de energia elétrica,1
Arte na Idade Média: Arte românica,1
Matemática Básica: Mínimo Múltiplo Comum,1
Biogeografia,1


In [119]:
p = data_step_clean_ques["topico"].value_counts().reset_index()
p[p['topico'] == 1]

,index,topico
2704,Senso Comum e Ideologia,1
2705,Filosofia Moderna: Utilitarismo: John Stuart Mill,1
2706,Ator,1
2707,"Clima, relevo e vegetação na formação da paisa...",1
2708,Estatística: Interpretação e desenvolvimento d...,1
...,...,...
3074,Matéria e energia: Fontes e tipos de energia: ...,1
3075,Arte na Idade Média: Arte românica,1
3076,Matemática Básica: Mínimo Múltiplo Comum,1
3077,Biogeografia,1


In [120]:
data_step_clean_ques.to_csv(r'C:/Users/Danilo/Desktop/Documentos Acadêmicos/TERA/TERAprojetos/Projeto final/BNCC/repo/bncc-classifier/data/curated/data_eda_treat_will.csv', index = False)
